In [ ]:
#%matplotlib inline
%matplotlib notebook
import numpy as np
import os

import strawb
import shapely.geometry
import cv2

import matplotlib.pyplot as plt

import strawb
import scipy.ndimage
import scipy.spatial
import scipy.interpolate

path = os.path.join(strawb.Config.repository_home, 'resources/image_cluster_search')

# Load images

In [ ]:
mounting_mod_dict = {
    'TUMPMTSPECTROMETER001': cv2.imread(os.path.join(path, 'TUMPMTSPECTROMETER001_mounting_mod.png'))[:,:,::-1],
    'TUMPMTSPECTROMETER002': cv2.imread(os.path.join(path, 'TUMPMTSPECTROMETER002_mounting_mod.png'))[:,:,::-1],
}

# gen. masked array for all devices
devices = list(mounting_mod_dict.keys())

for i in devices:
    img = np.ma.array(mounting_mod_dict[i],
                      mask=np.zeros_like(mounting_mod_dict[i])
                     )   

    # define the color threshold
    m = img[:,:,0] > 200
    m &= img[:,:,1] < 50
    m &= img[:,:,2] < 50

    img.mask |= m.reshape((*m.shape, 1))
    mounting_mod_dict[i] = img

## Plot images

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=len(mounting_mod_dict), squeeze=False, sharex=True, sharey=True)
print(ax.shape)

for i, dev_i in enumerate(mounting_mod_dict):
    ax[0, i].set_title(dev_i.replace('TUMPMTSPECTROMETER','PMTSpec'))
    
    ax[0, i].imshow(mounting_mod_dict[dev_i])
    ax[1, i].imshow(mounting_mod_dict[dev_i].filled((0,0,0)))
    
ax[0, 0].set_ylabel('mod.png')
ax[1, 0].set_ylabel('masked')

plt.tight_layout()

## Save the mask
strawb takes the mounting_mask.npz for the config

In [ ]:
# save the mask in separate file
file_name = os.path.abspath(os.path.join(strawb.Config.repository_home,
                                         'src/strawb/sensors/camera/mounting_mask.npz'))
np.savez(file_name, **{i: np.any(~mounting_mod_dict[i].mask, axis=-1) for i in mounting_mod_dict})

# for testing if written
file_test = np.load(file_name)
file_test.files

# Cut FOV

In [ ]:
ncols = 4
fig, ax = plt.subplots(ncols=ncols, nrows=len(mounting_mod_dict), squeeze=False, 
                       #sharex='row', sharey='row'
                      )
ax = ax.flatten()

c = 255//2

for i, dev_i in enumerate(mounting_mod_dict):
    cam_conf = strawb.camera.Config(dev_i)
    rect = cv2.minAreaRect(np.argwhere(cam_conf.mask_mounting))
    points = cv2.boxPoints(rect)
    print(dev_i, ' ', rect)

    img = mounting_mod_dict[dev_i]
    ax[ncols*i].set_ylabel(dev_i.replace('TUM', ''))
    ax[ncols*i].imshow(img)
    ax[ncols*i+1].imshow(img.filled((c,c,c)))
    
    cs = ax[ncols*i+1].contourf(~img.mask.any(axis=-1), 1, hatches=['xxx', ''], alpha=1, )
    # ------------------------------
    # New bit here that handles changing the color of hatches
    colors = ['maroon', 'red', 'darkorange', 'gold', 'forestgreen',
              'darkturquoise', 'dodgerblue', 'darkviolet']
    # For each level, we set the color of its hatch 
    for j, collection in enumerate(cs.collections):
    #     collection.set_edgecolor(colors[j % len(colors)])
        collection.set_edgecolor('C0')
        collection.set_facecolor('none')#(0,0,0,0))

    # Doing this also colors in the box around each level
    # We can remove the colored line around the levels by setting the linewidth to 0
    for collection in cs.collections:
        collection.set_linewidth(1.)
    
    img_cut = strawb.camera.tools.cut_fov(img, rect=rect)
    ax[ncols*i+2].imshow(img_cut.filled((c,c,c)))
    ax[ncols*i+3].imshow(strawb.camera.tools.equalization(img_cut.filled((c,c,c))))
    
ax[0].set_title('Modified')
ax[1].set_title('masked red')
ax[2].set_title('+ cropped')
ax[3].set_title('+ equalization')
plt.tight_layout()

# Get contour as polygon and simplify the polygon

In [ ]:
# Plot all contours for all devices. There should be only one contour per device
plt.figure()
for i, dev_i in enumerate(mounting_mod_dict):
    cam_conf = strawb.camera.Config(dev_i)
    contours, hierarchy = strawb.camera.tools.get_contours_simplify(cam_conf.mask_mounting)

    if len(contours) != 1:
        print(f"WARNING: {dev_i} doesn't has exacly 1 contour. Got: {len(contours)}")
    for j, c_j in enumerate(contours):
        plt.plot(*c_j.T, label=f'{dev_i.replace("TUM","")}_contour_{j}')
        
plt.legend()

# Modifiy picture by hand to mask mounting
you can export the images stored in LED_images.npz to a png. Modify the image that the mounting is red ([255,0,0]) e.g. with photoshop

In [ ]:
with np.load('LED_images.npz') as f:
        images = f['images']
        
# select one bright image
image = images[-1]
        
# True, to export the image
if False:
    # Write image as png to modify it that the mounting is red ([255,0,0]) e.g. with photoshop
    # imwrite needs 16bit uint and BGR instead of RGB <-> [:,:,::-1]
    cv2.imwrite('mounting.png', (image * 2**16).astype(np.uint16)[:,:,::-1])

In [ ]:
images_list = list(images.astype(np.float32))

images_list_show = [images_list[1], images_list[3], images_list[-1]]
fig, ax = plt.subplots(figsize=(10,5), ncols=len(images_list_show), sharex=True, sharey=True, squeeze=False)
ax = ax.flatten()
for i, img_i in enumerate(images_list_show):
    ax[i].imshow(strawb.camera.tools.equalization(img_i))

# Get position of module above

In [ ]:
# lower limits of the RGB valuse [red, green, blue]
limit = np.ones(3) * .1

box_list = []
images_list_show = [images_list[1], images_list[3]]

fig, ax = plt.subplots(figsize=(10,5), ncols=len(images_list_show), sharex=True, sharey=True, squeeze=False)
ax = ax.flatten()

for i, img_i in enumerate(images_list_show):
    
    mask = img_i[:,:,0] > limit[0]
    mask &= img_i[:,:,1] > limit[1]
    mask &= img_i[:,:,2] > limit[2]

    box = cv2.minAreaRect(np.argwhere(mask))
    box_list.append(box)
    points = cv2.boxPoints(box)

    ax[i].imshow(strawb.camera.tools.equalization(img_i))

    ax[i].scatter(*box[0][::-1], alpha=.5, color='w')
    ax[i].plot(*strawb.tools.connect_polar(points).T[::-1], alpha=.5, color='w' )

# added to strawb.camera.Config.position_dict: device_code: {'module' : ...}
print(f'Mean position of {len(images_list_show)} pictures')
print(f"'module' : {np.array([i[0] for i in box_list]).mean(axis=0)}")

# Plot imges with labels

In [ ]:
# plot
cmap = strawb.tools.cmap_manipulator('Reds', alpha_min=0)
fig, ax = plt.subplots(figsize=(10,5), ncols=len(mounting_mod_dict),
                       sharex=True, sharey=True, squeeze=False)
ax = ax.flatten()

for i, dev_i in enumerate(mounting_mod_dict):
    cam_conf = strawb.camera.Config(dev_i)

    img = mounting_mod_dict[dev_i]
    ax[i].imshow(img.filled(0))
    # plt.imshow(cv2.cvtColor(image.astype(np.float32), cv2.COLOR_BGR2GRAY), cmap='gray_r')
    # plt.imshow(mask, label='2D Mask', cmap=cmap, alpha=mask.astype(int))

    # for i, c_i in enumerate(contours):
    #     plt.plot(*c_i.T, '-', label=f'cv2 contours {i}')

    contours, hierarchy = strawb.camera.tools.get_contours_simplify(cam_conf.mask_mounting)
    print(dev_i, img.shape)
    for j, c_j in enumerate(contours):
        ax[i].plot(*c_j.T, ':', label=f'FoV boundary', lw=3, color='w')

    # cs = plt.contourf(mounting_mask, levels=1, hatches=['/', ''], alpha=1)
    # # ------------------------------
    # # New bit here that handles changing the color of hatches
    # colors = ['maroon', 'red', 'darkorange', 'gold', 'forestgreen',
    #           'darkturquoise', 'dodgerblue', 'darkviolet']
    # # For each level, we set the color of its hatch 
    # for i, collection in enumerate(cs.collections):
    # #     collection.set_edgecolor(colors[i % len(colors)])
    #     collection.set_edgecolor('gray')
    #     collection.set_facecolor('none')#(0,0,0,0))

    # # Doing this also colors in the box around each level
    # # We can remove the colored line around the levels by setting the linewidth to 0
    #     collection.set_linewidth(1.)

    ax[i].plot(*cam_conf.data_cable, label='Data Cable')
    ax[i].plot(*cam_conf.steel_cable, label='Steel Cable')
    ax[i].plot(*cam_conf.position_module, 'o', label='Module')

    ax[i].legend(loc=1, ncol=2)
# min_xy = points.min(axis=0).astype(int)
# max_xy = points.max(axis=0).astype(int)
# plt.ylim(min_xy[0]-10, max_xy[0]+10)
# plt.xlim(min_xy[1]-10, max_xy[1]+10)
# plt.gca().invert_yaxis()
plt.tight_layout()